## Algoritmia
### Práctica 2
El objetivo de esta práctica es definir clases y realizar implementaciones básicas del tipo grafo.

Se pide la implementación de las clases y/o funciones que aparecen a continuación. 

Las instrucción "pass" que aparecen en el cuerdo de las clases o funciones, se debe sustituir por la implementación adecuada. 

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests. 

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Clase abstracta para Grafos

In [1]:
from abc import ABCMeta, abstractmethod

class GrafoAbstracto(metaclass=ABCMeta):
    """Clase abstracta para trabajar con Grafos."""
    
    def __init__(self, dirigido = False):
        """Constructor. El argumento indica si el grafo es dirigido"""
        self._dirigido = dirigido

    def dirigido(self):
        """Indica si el grafo es o no dirigido"""
        return self._dirigido

    @abstractmethod
    def __len__( self ):
        """Número de nodos del grafo."""
        return 

    @abstractmethod
    def num_arcos(self):
        """Devuelve el número de arcos"""

    @abstractmethod       
    def inserta(self, nodo, destino = None, etiqueta = 1):
        """
        Inserta un nodo al grafo (si destino es None) o un arco.
        Si el arco ya existía se actualiza su etiqueta.
        Si alguno de los nodos del arco no está en el grafo, se inserta.
        Se supone que None no es una etiqueta válida.
        """

    @abstractmethod       
    def __contains__(self, nodo):
        """Indica si el nodo está en el grafo."""      
        
    @abstractmethod        
    def __getitem__(self, arco):
        """Dado un arco (un par de nodos) devuelve la etiqueta si el arco está
        en el grafo, en caso contrario devuelve None"""

    @abstractmethod        
    def __iter__(self):
        """Iterador sobre los nodos del grafo"""

    @abstractmethod
    def vecinos(self, origen):
        """Devuelve un iterable de los pares (destino,etiqueta) para un nodo 
        origen dado"""

### Implementación basada en matrices de adyacencia
Realizamos una implementación basada en [matrices de adyacencia](https://youtu.be/t-FHxHnUEoc)

In [2]:
class GrafoMatriz(GrafoAbstracto):
    """
    Implementación del tipo Grafo utilizando una matriz de adyacencia para 
    almacenar la información de los arcos.
    La matriz podría ser una lista de lista.
    """
    
    
    def __init__(self, dirigido = False):
        """Constructor. El argumento indica si el grafo es dirigido"""
        self._dirigido = dirigido
        self._nnodos=0
        self._narcos=0
        self._d=dict()
        self._l=[]

    def dirigido(self):
        """Indica si el grafo es o no dirigido"""
        return self._dirigido
    
    def __len__(self):
        return self._nnodos
        #return 6
    
    
    def num_arcos(self):
        """Devuelve el número de arcos"""
        return self._narcos

          
    def inserta(self, nodo, destino = None, etiqueta = 1):
        """
        Inserta un nodo al grafo (si destino es None) o un arco.
        Si el arco ya existía se actualiza su etiqueta.
        Si alguno de los nodos del arco no está en el grafo, se inserta.
        Se supone que None no es una etiqueta válida.
        """
        if destino is None:
            tpl=(nodo)
            self._d.setdefault(tpl,None)
            self._l.append(nodo)
            self._nnodos=self._nnodos+1
        else:
            tpl=(nodo,destino)
            self._d.setdefault(tpl,etiqueta)
            
            if nodo not in self._l:
                self._l.append(nodo)
                self._nnodos=self._nnodos+1
            if destino not in self._l:
                self._l.append(destino)
                self._nnodos=self._nnodos+1
                
            self._narcos=self._narcos+1
            if self.dirigido() is False:
                tpla=(destino,nodo)
                self._d.setdefault(tpla,etiqueta)
            
    def __contains__(self, nodo):
        """Indica si el nodo está en el grafo.""" 
        #Es la funcion que se llama en if nodo in grafo
        if nodo in self._d:
            return True
        else:
            return False
        
           
    def __getitem__(self, arco):
        """Dado un arco (un par de nodos) devuelve la etiqueta si el arco está
        en el grafo, en caso contrario devuelve None"""
        #assert grafo[nodo_m, nodo_n] == etiqueta
        no=arco[0]#nodo origen
        nd=arco[1]#nodo destino
        tpl=(no,nd)
        ret=self._d.get(tpl)
        return ret

            
    def __iter__(self):
        """Iterador sobre los nodos del grafo"""
        for ret in self._d:#ret va ser una tupla [norigen,ndestino], es decir, las claves
            if self._d[ret] is None:
                yield ret
            
    def vecinos(self, origen):
        """Devuelve un iterable de los pares (destino,etiqueta) para un nodo 
        origen dado"""
        tpl=[]
        for ret in self._d:
            if origen in ret[0]:
                t=(ret[1],self._d.get(ret))
                tpl.append(t)
        return tpl
            
    pass

### Implementación basada en listas de adyacencia
Realizamos una implementación basada en [listas de adyacencia](https://youtu.be/7cXY3ztIGjs)

In [3]:
class GrafoListas(GrafoAbstracto):
    """
    Implementación del tipo Grafo utilizando listas de adyacencia.
    Cada nodo tiene asociada una 'lista' con la información de los arcos que
    salen del nodo. 
    Dicha lista no tinene que ser necesariamente del tipo 'list' de Python.
    """
    

    def __init__(self, dirigido = False):
        """Constructor. El argumento indica si el grafo es dirigido"""
        self._dirigido = dirigido
        self._arcos=0
        self._nodos=0
        self._diccionario ={}
    
    def dirigido(self):
        """Indica si el grafo es o no dirigido"""
        return self._dirigido
    
    def __len__(self):
        return len(self._diccionario)
    
    def num_arcos(self):
        return self._arcos
    
    def inserta(self, nodo, destino = None, etiqueta = 1):
        """
        Inserta un nodo al grafo (si destino es None) o un arco.
        Si el arco ya existía se actualiza su etiqueta.
        Si alguno de los nodos del arco no está en el grafo, se inserta.
        Se supone que None no es una etiqueta válida.
        """
        
        if not nodo in self._diccionario:#Si el nodo no esta ya metido en el dicionario, lo metemos.
            self._diccionario[nodo]=[]
        if not destino  is None:
            if not destino in self._diccionario: #Si el destino no es None, lo metemos
                self._diccionario[destino]=[]

            self._diccionario[nodo].append((destino,etiqueta)) #añadimos al nodo la tupla (destino,etiqueta),se crea el arco                
            if self.dirigido() is False:
                self._diccionario[destino].append((nodo,etiqueta))#añadimos al destino el arco con el nodo origen y la etiqueta

            self._arcos=self._arcos+1
        

    def __contains__(self, nodo):
        """Indica si el nodo está en el grafo."""      
        if nodo in self._diccionario:
            return True
        else:
            return False
        
    def __getitem__(self, arco):
        """Dado un arco (un par de nodos) devuelve la etiqueta si el arco está
        en el grafo, en caso contrario devuelve None"""
        ori=arco[0]
        fin=arco[1]
        if ori in self._diccionario:
            for n in self._diccionario[ori]:
                if fin == n[0]:
                    return n[1]
        
    def __iter__(self):
        """Iterador sobre los nodos del grafo"""
        return iter(self._diccionario)
    
    def vecinos(self, origen):
        """Devuelve un iterable de los pares (destino,etiqueta) para un nodo 
        origen dado"""
        for n in self._diccionario[origen]: #Recorre todos los valores unidos al origen
            yield n                         #Devuelve un primer valor y vuelve a devolver el siguiente hasta que acaba

    pass

### Casos de prueba

In [4]:
def test_grafo(grafo):
    """Función que prueba las funciones sobre grafos. Espera un grafo vacío."""

    num_final = 10  # número de nodos del grafo final
    num_arcos = 0
    conjunto_nodos = set()  # nodos que debería tener el grafo
    conjunto_arcos = set()  # arcos que debería tener el grafo
    
    # Insertamos nodos y arcos en el grafo, comprobando que la información es 
    # coherente con lo que tenemos en conjunto_nodos y conjunto_arcos
    for n in range(num_final):
        assert len(grafo) == n
        nodo_n = "n" + str(n)
        grafo.inserta(nodo_n)
        conjunto_nodos.add(nodo_n)
        assert nodo_n in grafo 
        assert n not in grafo
        
        for m in range(n):
            nodo_m = "n" + str(m)
            etiqueta = num_final * n + m
            grafo.inserta(nodo_m, nodo_n, etiqueta)
            conjunto_arcos.add((nodo_m, nodo_n, etiqueta))
            num_arcos += 1
            assert num_arcos == grafo.num_arcos()
            assert grafo[nodo_m, nodo_n] == etiqueta
            if grafo.dirigido():
                assert grafo[nodo_n, nodo_m] == None
            else:
                assert grafo[nodo_n, nodo_m] == etiqueta
                conjunto_arcos.add((nodo_n, nodo_m, etiqueta))
       
    # Recorremos comproabando los nodos y para cada nodo sus vecinos
    for nodo_n in grafo:
        assert nodo_n in conjunto_nodos, nodo_n
        conjunto_nodos.remove(nodo_n)
        for nodo_m, etiqueta in grafo.vecinos(nodo_n):
            assert (nodo_n, nodo_m, etiqueta) in conjunto_arcos
            conjunto_arcos.remove((nodo_n, nodo_m, etiqueta))       
   
    # Comprobamos que hemos recorrido todos los nodos y arcos
    assert len(conjunto_nodos) == 0,len(conjunto_nodos)
    assert len(conjunto_arcos) == 0,len(conjunto_arcos)

     

In [5]:
if __name__ == "__main__":     
    test_grafo(GrafoMatriz(False))
    test_grafo(GrafoMatriz(True))
    print("OK")

OK


In [6]:
if __name__ == "__main__":     
    test_grafo(GrafoListas(False))
    test_grafo(GrafoListas(True))
    print("OK")

OK


In [7]:
"""
Con %timeit podemos ver el tiempo necesario para ejecutar una línea.
Puede ejecutarla múltiples veces para tener una mejor estimación.
Con %%timeit obtenemos el tiempo de ejecución de una celda.
"""
if __name__ == "__main__":  
    %timeit test_grafo(GrafoMatriz(False))
    %timeit test_grafo(GrafoMatriz(True))
    %timeit test_grafo(GrafoListas(False))
    %timeit test_grafo(GrafoListas(True))

The slowest run took 5.23 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 384 µs per loop
1000 loops, best of 3: 257 µs per loop
1000 loops, best of 3: 252 µs per loop
10000 loops, best of 3: 198 µs per loop
